In [1]:
import pandas as pd
import numpy as np

import pickle
import json

In [2]:
test_df = pd.read_csv('./output/中間データ/test_preprocessed.csv')
test_df.head()

,datetime,y,week,soldout,name,kcal,payday,weather,temperature,y_1lag,...,y_rolling_max_14,y_rolling_min_14,day,kcal_missing_flag,curry,popular,unpopular,fun,month,temp
0,2014-10-01,NaN,0.416215,1,メンチカツ,-0.079503,0.0,NaN,0.348718,40.0,...,115.0,29.0,-1.725325,0,0,1,0,0,21.140909,-0.275175
1,2014-10-02,NaN,-0.871488,0,バーベキューチキン,-0.446438,0.0,NaN,1.189276,NaN,...,NaN,NaN,-1.611442,0,0,0,0,0,21.140909,0.806914
2,2014-10-03,NaN,-0.843645,0,豚肉のマスタード焼き,-1.180308,0.0,NaN,2.279729,NaN,...,NaN,NaN,-1.497560,0,0,0,0,0,21.140909,2.210706
3,2014-10-06,NaN,2.197171,1,麻婆春雨,-1.547242,0.0,NaN,0.644049,NaN,...,NaN,NaN,-1.155911,0,0,0,0,0,21.140909,0.105018
4,2014-10-07,NaN,0.088051,0,厚揚げ肉みそ炒め,0.654367,0.0,NaN,0.780356,NaN,...,NaN,NaN,-1.042028,0,0,0,0,0,21.140909,0.280492


In [3]:
X_test = test_df.drop(columns=['y', 'name', 'datetime'])

In [4]:
with open('./output/モデル/lgb_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

new_y_pred = loaded_model.predict(X_test)

#### 予測結果の出力

In [5]:
result_df = pd.read_csv('./input/sample.csv', header=None)

In [6]:
train_df = pd.read_csv('./output/中間データ/train_preprocessed.csv')
test_df['index_new'] = test_df.index + train_df.index.max() + 1
xs = test_df['index_new'].values

# トレンド係数の読み込み
with open('./output/中間データ/trend_params.json', 'r') as f:
    trend_params = json.load(f)

a1, b1, c1 = trend_params['a'], trend_params['b'], trend_params['c']
trend_pred = a1 * np.exp(-b1 * xs) + c1

y_pred = new_y_pred + trend_pred

In [7]:
result_df.iloc[:, 1] = y_pred

/var/folders/sb/t91v_nzj05sbsfnhcjb832xr0000gn/T/ipykernel_13773/3961981253.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[54.32691785 53.22476671 53.09027991 54.08241637 53.47690308 53.62531415
 52.41661935 56.09683358 53.08984523 53.54756091 51.89875928 51.78893827
 52.79721549 52.53009165 52.82060629 51.47219867 51.96908923 52.18173765
 52.46626532 51.9345086  50.80747008 50.73942243 51.81072652 52.12526851
 50.55487123 53.58114373 51.21627061 51.54768546 51.43765211 50.10939676
 49.89431514 51.20015216 51.05029129 50.9890347  49.48499966 52.07445457
 50.52405642 50.3505102  49.14819234 49.05508314]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result_df.iloc[:, 1] = y_pred


In [8]:
result_df.to_csv('output/予測結果/予測結果.csv', index=False)